# Start


Quick solution for tensorflow module not found


In [ ]:
# !pip install tensorflow

In [ ]:
import os
import shutil

import cv2
import kagglehub
import keras_tuner as kt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.layers import (
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    InputLayer,
    MaxPooling2D,
    Rescaling,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image

## Import data


In [ ]:
def import_raw_data():
    # Load dataset
    path = kagglehub.dataset_download("arashnic/faces-age-detection-dataset")

    # Define path to files
    faces_path = os.path.join(path, "faces")
    train_path = os.path.join(faces_path, "Train")

    # Create directory for raw data
    export_path = "../data/raw/faces"
    os.makedirs(export_path, exist_ok=True)

    # Copy files to raw data directory
    shutil.copytree(train_path, os.path.join(export_path, "images"), dirs_exist_ok=True)
    shutil.copy(os.path.join(faces_path, "train.csv"), os.path.join(export_path, "train.csv"))

    export_path = "../data/raw"


if not os.path.exists("../data/raw/faces"):
    import_raw_data()

-   Faces_02 has no csv file for classification so we are gonna skip it


## Definitons


In [ ]:
df_train = pd.read_csv("../data/raw/faces/train.csv")

# 4.1


## A


-   EDA a data preprocessing pre Vami vybrané charakteristiky z datasetu


### EDA


In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.duplicated().sum()

In [ ]:
df_train.isnull().sum()

In [ ]:
image_dir = "../data/raw/faces/images"
num_files = len([name for name in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, name))])
print(f"Number of images: {num_files}")

In [ ]:
print(df_train["Class"].value_counts())
df_train["Class"].value_counts().plot(kind="bar")
plt.xticks(rotation=45)
plt.show()

-   There are no nulls
-   There are no duplicates
-   Number of images is same as number of data in cvs file


### Data preprocessing


In [ ]:
# Resize images and save them into categorical directories
def process_images(images, target_size, color_mode):
    output_dir = f"../data/processed/faces/{color_mode}/{target_size[0]}x{target_size[1]}"

    young_dir = os.path.join(output_dir, "young")
    middle_dir = os.path.join(output_dir, "middle")
    old_dir = os.path.join(output_dir, "old")

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(young_dir, exist_ok=True)
    os.makedirs(middle_dir, exist_ok=True)
    os.makedirs(old_dir, exist_ok=True)

    for _, row in images.iterrows():
        img_name = row["ID"]
        img_path = os.path.join("../data/raw/faces/images", img_name)
        img = image.load_img(img_path, target_size=target_size, color_mode=color_mode)

        if row["Class"].upper() == "YOUNG":
            img.save(os.path.join(young_dir, img_name))
        elif row["Class"].upper() == "MIDDLE":
            img.save(os.path.join(middle_dir, img_name))
        elif row["Class"].upper() == "OLD":
            img.save(os.path.join(old_dir, img_name))


# Get low quality images
def get_low_quality_images(images, min_width=15, min_height=15):
    image_dir = "../data/raw/faces/images"
    low_quality_images = []

    for image_id in images["ID"]:

        image_path = os.path.join(image_dir, image_id)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if img.shape[0] < min_width or img.shape[1] < min_height:
            low_quality_images.append(image_id)
            continue

        compressed = cv2.resize(img, None, fx=0.5, fy=0.5)
        compressed = cv2.resize(compressed, (img.shape[1], img.shape[0]))
        score, _ = ssim(img, compressed, multichannel=True, full=True)
        if score < 0.88:
            low_quality_images.append(image_id)

    return low_quality_images


df_processed = df_train.copy()
if os.path.exists("../data/processed"):
    shutil.rmtree("../data/processed")

os.makedirs("../data/processed/faces", exist_ok=True)

# Get low quality images
print("Getting low quality images")
low_quality_images = get_low_quality_images(df_processed)
print(f"Number of low quality images: {len(low_quality_images)}")
print(f"Percentage of low quality images: {len(low_quality_images) / len(df_processed) * 100:.2f}%")

# Remove low quality images
df_processed = df_processed[~df_processed["ID"].isin(low_quality_images)]

# Process images
print("Processing images, 64x64, grayscale")
process_images(df_processed, (64, 64), "grayscale")

print("Processing images, 64x64, rgb")
process_images(df_processed, (64, 64), "rgb")

print("Processing images, 128x128, grayscale")
process_images(df_processed, (128, 128), "grayscale")

print("Processing images, 128x128, rgb")
process_images(df_processed, (128, 128), "rgb")

## B


-   Zdôvodnite výber ML/DL metód vzhľadom na Vami vybraný dataset pre 4.2


# 4.2


## A


-   Modeluje Vami tie vybrané charakteristiky pomocou vhodných ML/DL
    metód. Výsledok modelovania je najlepší model.


#### Definitions


In [ ]:
BATCH_SIZE = 64
EPOCHS = 10


def get_dataset(size, color_mode):
    # Load dataset
    ds = tf.keras.preprocessing.image_dataset_from_directory(
        f"../data/processed/faces/{color_mode}/{size}x{size}",
        shuffle=True,
        image_size=(size, size),
        batch_size=BATCH_SIZE,
    )

    # Split dataset
    data_size = len(ds)
    train_split = 0.7
    val_split = 0.2
    test_split = 0.1

    train_size = int(train_split * data_size)
    val_size = int(val_split * data_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    # Cache, shuffle and prefetch
    buffer_size = len(train_ds) * BATCH_SIZE
    buffer_size

    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

    return train_ds, val_ds, test_ds


def get_input_shape(train_ds):
    for batch, _ in train_ds.take(1):
        return batch.shape

In [ ]:
def show_accuracy(history):
    plt.plot(history.history["accuracy"], label="Training Accuracy")
    plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
    plt.title("Model accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Validation"], loc="upper left")
    plt.show()


def show_loss(history):
    plt.plot(history.history["loss"], label="Training Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title("Model loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Validation"], loc="upper left")
    plt.show()


def show_evaluation(model, val_ds, history):
    loss, acc = model.evaluate(val_ds, batch_size=BATCH_SIZE)
    print(f"Loss: {round(loss, 3)}, Acc: {round(acc*100, 3)}%")
    show_accuracy(history)
    show_loss(history)

#### Main Model


In [ ]:
train_ds, val_ds, test_ds = get_dataset(64, "rgb")
input_shape = get_input_shape(train_ds)

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
model = Sequential(
    [
        # Preprocessing layers
        InputLayer(shape=input_shape[1:]),
        Rescaling(1.0 / 255),
        # Input and first conv block
        Conv2D(32, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        # Second conv block
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        # Third conv block
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        # Flatten and dense layers
        Flatten(),
        Dense(128, activation="relu"),
        Dense(3, activation="softmax"),
    ]
)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    batch_size=BATCH_SIZE,
    validation_batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
)

In [ ]:
show_evaluation(model, val_ds, history)

#### Test Model


In [ ]:
train_ds, val_ds, test_ds = get_dataset(64)
input_shape = get_input_shape(train_ds)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
def build_model(hp):
    model = Sequential(
        [
            # Input and preprocessing layers
            InputLayer(shape=input_shape[1:]),
            Rescaling(1.0 / 255),
            # First conv block - tune filters and kernel size
            Conv2D(
                32,
                hp.Choice("conv1_kernel", values=[3, 5]),
                activation="relu",
            ),
            MaxPooling2D((2, 2)),
            # Second conv block - tune filters and kernel size
            Conv2D(
                64,
                hp.Choice("conv2_kernel", values=[3, 5]),
                activation="relu",
            ),
            MaxPooling2D((2, 2)),
            # Third conv block - tune filters and kernel size
            Conv2D(
                128,
                hp.Choice("conv3_kernel", values=[3, 5]),
                activation="relu",
            ),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation="relu"),
            Dropout(0.5),
            Dense(3, activation="softmax"),
        ]
    )

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model


# Create tuner
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=20,
    factor=3,
    directory="tuning_dir",
    project_name="age_classification",
    overwrite=True,
)

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)


# Search for best hyperparameters
tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stopping])

# Get best hyperparameters and build final model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = build_model(best_hps)

# Train final model with best hyperparameters
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stopping])

# Evaluate model
show_evaluation(model, val_ds, history)

#### Test Model 2


In [ ]:
train_ds, val_ds, test_ds = get_dataset(64)
input_shape = get_input_shape(train_ds)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
model = Sequential(
    [
        # Preprocessing layers
        InputLayer(shape=input_shape[1:]),
        Rescaling(1.0 / 255),
        # Input and first conv block
        Conv2D(32, (5, 5), activation="relu"),
        MaxPooling2D((2, 2)),
        # Second conv block
        Conv2D(64, (5, 5), activation="relu"),
        MaxPooling2D((2, 2)),
        # Third conv block
        Conv2D(128, (5, 5), activation="relu"),
        MaxPooling2D((2, 2)),
        # Flatten and dense layers
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(3, activation="softmax"),
    ]
)


model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    batch_size=BATCH_SIZE,
    validation_batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
)

In [ ]:
show_evaluation(model, val_ds, history)

#### Test Model 3


In [ ]:
train_ds, val_ds, test_ds = get_dataset(64)
input_shape = get_input_shape(train_ds)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
model = Sequential(
    [
        # Preprocessing layers
        InputLayer(shape=input_shape[1:]),
        Rescaling(1.0 / 255),
        # Input and first conv block
        Conv2D(32, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        # Second conv block
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        # Third conv block
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        # Flatten and dense layers
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(3, activation="softmax"),
    ]
)


model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    batch_size=BATCH_SIZE,
    validation_batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
)

In [ ]:
show_evaluation(model, val_ds, history)

## B


-   Zhodnotíte Váš prístup a získaný výsledok


In [ ]:
def build_model(hp):
    model = Sequential(
        [
            # Input and preprocessing layers
            InputLayer(shape=input_shape[1:]),
            Rescaling(1.0 / 255),
            # First conv block - tune filters and kernel size
            Conv2D(
                hp.Int("conv1_filters", min_value=8, max_value=32, step=8),
                hp.Choice("conv1_kernel", values=[3, 5]),
                padding="same",
                activation="relu",
            ),
            MaxPooling2D((2, 2)),
            # Second conv block
            Conv2D(
                hp.Int("conv2_filters", min_value=16, max_value=64, step=16),
                hp.Choice("conv2_kernel", values=[3, 5]),
                padding="same",
                activation="relu",
            ),
            MaxPooling2D((2, 2)),
            # Third conv block
            Conv2D(
                hp.Int("conv3_filters", min_value=32, max_value=128, step=32),
                hp.Choice("conv3_kernel", values=[3, 5]),
                padding="same",
                activation="relu",
            ),
            MaxPooling2D((2, 2)),
            # Dense layers
            Flatten(),
            Dense(hp.Int("dense_units", min_value=32, max_value=128, step=32), activation="relu"),
            Dropout(hp.Float("dropout", min_value=0.0, max_value=0.5, step=0.1)),
            Dense(3, activation="softmax"),
        ]
    )

    # Tune learning rate
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=["accuracy"],
    )

    return model


# Create tuner
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="tuning_dir",
    project_name="age_classification",
)

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# Search for best hyperparameters
tuner.search(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stopping])

# Get best hyperparameters and build final model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = build_model(best_hps)

# Train final model with best hyperparameters
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stopping])